In [ ]:
# Считайте набор данных

df_fraud = pd.read_csv
df_empl = pd.read_csv('employee.csv')

## OneHot

In [ ]:
# Закодируйте категориальные признаки числами 0 и 1 с помощью OneHotEncoder
# Выделите отдельные датасеты с закодированными признаками
#   train -> fit_transform
#   test -> transform

empl_encoder = OneHotEncoder(sparse_output=False, drop='first').set_output(transform='pandas')

X_empl_train_onehot = empl_encoder.fit_transform(X_empl_train[empl_cat_feat])
X_empl_test_onehot = empl_encoder.transform(X_empl_test[empl_cat_feat])

# Methods of Generation and Load data

5. WITH [make_circles](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_circles.html) 

In [ ]:
from sklearn.datasets import make_circles

#make circles

X_circles, y_circles = make_circles(n_samples=1000, factor=0.1, noise=0.3, random_state=RANDOM_STATE)
X_circles = pd.DataFrame(X_circles)
X_circles.columns = ['x1', 'x2']


Function [choice](https://numpy.org/doc/stable/reference/random/generated/numpy.random.choice.html)
сгенерируйте 5 бутстрэп-подвыброк

In [ ]:
# Не забудьте зафиксировать RANDOM_STATE (с помощью переменной rng)
rng = np.random.RandomState(RANDOM_STATE)

k = 5 # Число деревьев в ансамбле (число бутстрэп-подвыброк)

train_N = len(X_titanic_train) # Размер обучающей выборки

train_idx = list(range(train_N)) # Список с индексами наблюдений обучающей выборки

# Выполните случайный выбор индексов train_idx с повторениями
titanic_bootstraps = [rng.choice(train_idx, size=train_N, replace=True) for _ in range(k)]

## 6. RNG

In [ ]:
# Сгенерируйте простой набор данных для регрессии

rng = np.random.RandomState(RANDOM_STATE)
noise = rng.normal(0, 0.18, 1000)
X = pd.DataFrame({'x': np.linspace(1, 10, 1000)})
y_true = -0.3 * X['x'] ** 0.5 * np.cos(np.pi * X['x'] / 4)
y_true = np.array(y_true)
y = y_true + noise